# Kinetic Proofreading Animation
2019
Author: Chen Zuckerman Nelson  (nelson@physics.upenn.edu)

Python: 3.8   ChenCode4-kproofBackend.ipynb

Description: VPython animation to display the simulation results from Code 3. It reads the data files kproof_HN.npz or kproof_realistic.npz emitted by Code 3. If the flag "povray" is set True, then this code writes a lot of .pov files (ray-traced images of each frame). Use "POVRay Unofficial.app" to convert to .png. Then, use 
ffmpeg to compile .png files into movie.

Example ffmpeg command: <br>
ffmpeg -r 15 -i "/C:/pathname/%04d_realistic.png" -pix_fmt yuv420p realistic_150s.mp4 

The number 15 sets the framerate of the compiled movie. It should match the framerate chosen when creating the animation using the code below. The "%04d" in the filename tells ffmpeg to look for files with the name format 0001_realistic.png, 0002_realistic.png, ... and so on. "realistic_150s.mp4" is the chosen name of the movie.  
<br> 
Generates Video 3, 4 and 5 of "Stochastic Simulation to Visualize Gene Expression and Error Correction in Living Cells"<br>

tRNA-amino acid (AA)-GTP complexes appear from solution<br>
red = wrong; green = right<br>
GTP = dark color; GDP = bright color<br>
GTP = cylinder; GDP = sphere; <br>
GTP hydrolysis = shift right and deepening color<br>
rejected tRNA-AA-GxP appear briefly ghosted then disappear<br>
AA incorporation shifts the nascent chain over to the right.


Preparations; set appearance parameters; load data:

In [1]:
from vpython import *
import numpy as np

myopaci = 0.2               # appearance: opacity of arriving and leaving objects
povray = False              # whether to generate POVray files
fracshow = 1.00             # to show only a portion of data (e.g. 0.94)
framerate = 15              # frame rate in Hz, so 30 means 33ms per frame
duration = framerate*60*3   # desired duration of movie [total viewing time in seconds]
rejectduration = 5          # how long to display rejected tRNA-AA-GxP (in frames)

# some geometrical parameters about display:
mysphererad = 0.15          # blob size representing amino acids
hydrolysis_sh = mysphererad-0.1 # rightward shift after hydrolysis

if povray: import povexport; myopaci=0.5            # optional for nicer POVray rendering
    
"""Get incoming data generated by simulation engine ChenCode3-riboProof.py. Change to 
  kproof_HN.npz for simulation with Hopfield-Ninio parameters. Use
  kproof_realistic.npz for simulation with more realistic model."""
data = np.load('kproof_realistic.npz')  
print("Data generated ", data['now']," in simulation type ", data['simtype'])
# Extract data from the structure just loaded:
ts = data['ts'][0]          # simulation times of transitions
states = data['states'][0]  # sequence of states:  states[j] is state exited at ts[j]. 
                            # states[j+1] is state entered at ts[j].
simtype = str(data['simtype'])
Nstates = len(states) - 1
Nframes = int(duration*framerate)
print(Nstates," states displayed in ",Nframes,' frames.' )
scratch_dir = "desired directory to save .pov files" # choose something convenient
outfile_name = "{:05d}_"+simtype+".pov"   # prepend scratch_dir if desired, {:05d} creates
                                          # with name format 00001_simtype.pov, 00002_simtype.pov, etc.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data generated  2018-07-05 15:29:22.915828  in simulation type  Banerjee
138  states displayed in  150  frames.


Identify key events:<br>
A video frame starts at view-time $m(\Delta t)$ and ends at $(m+1)(\Delta t)$, where $m$ is the frame index and $\Delta t$ is the view time per frame. We wish to identify, and display, the state that is current at $m(\Delta t)$ (i.e. the most recent state change prior to that), along with two kinds of decoration: "incoming" and "outgoing" objects.<br>
View time tv is time as seen by viewer, distinct from "simulation time" ts, which is what the simulation code riboProof outputs. State states$_j$ ends (and state states$_{j+1}$ begins) at simulation time ts$_j$.<br>
The correspondence is that view time tv$_j$ equals $j*t_{min}+K*$ts$_j$. Here $t_{min}$ is a minimal delay added to every waiting time and $K$ is a constant chosen to achieve the specified overall duration of the animation.<br>
We first step through the simulation times to find the index Jm$_m$ such that $m(\Delta t)$ just exceeds tvtemp[Jm$_m$] (tvtemp is the same as tv, but with a dummy entry appended at the beginning so that states$_j$ is entered at tvtemp$_j$). Then, states$_{Jm_m}$ will be the state displayed in the frame that starts at $m(\Delta t)$ and ends at $(m+1)(\Delta t)$.

In [2]:
Deltat = 1/framerate    # view time per frame
# for easier visualization, we can specify a minimal waiting time between events:
tmin = 0.50*Deltat      # 0 means don't adjust waiting times; nonzero adds a minimal wait between each transition time
K = (duration - Nstates*tmin)/ts[-1] # scaling constant that connects simulation time to view time
if K<=0: error('need longer duration') # minimum duration needed is Nstates*tmin
tv = np.arange(Nstates)*tmin + K*ts  # view time of each transition
tvtemp = np.insert(tv,0,-1.)         # insert dummy entry at beginning (-1.0) to make loop below terminate
                                     # also makes it such that states[j] is entered at tvtemp[j]
Jm = -1*np.ones(Nframes+1, dtype='int') # will contain index of state that
                                        # is current at the beginning of each video frame
j = Nstates                          # work backward from the end to the beginning: last state to first state   
for m in range(Nframes,-1,-1):       # work backward from the end to the beginning: last frame to first frame
    viewtime = m/framerate
    while tvtemp[j] > viewtime: 
        j = j-1                      # keep stepping the state back to find one that precedes this frame
    Jm[m] = j                        # prepend another entry then repeat for previous frame till done

Jm = np.append(Jm, Nstates-1)        # add last state to end of Jm since displaying arriving tRNAs 
                                     # requires looking ahead one state

Set up objects, many initially invisible:

In [3]:
scene = canvas(title='KP')
scene.userzoom = False          # disables ability to zoom in and out on canvas with mouse scroll
scene.forward = vector(0,.4,-1) # camera viewpoint
# represent ribosome by a blue tabletop
static = box(pos=vector(0,0,0), size=vector(mysphererad*0.9*25*1.5,1,.2), color=color.blue)
# represent the bound tRNA-GTP complex by a cylinder, with dark color 
bindsite = vector(-.15,0,.1) # binding site of tRNA-GTP complex
boundT = cylinder(pos=bindsite+vector(0,0,mysphererad*1.5), axis=vector(0,0,.17),\
                  radius=mysphererad*.85)
boundT.visible = False # initially none, True when binding occurs
# after a tRNA-GTP is rejected, briefly show it as ghost
rejectT = cylinder(pos=bindsite+vector(0.5*mysphererad, 0.3*mysphererad, 1.7*mysphererad), \
                   axis=vector(0,0,.17), radius=mysphererad*.80, opacity=myopaci)
rejectT.visible = False # initially none, True when tRNA-GTP is rejected
# represent the bound tRNA-GDP complex by a sphere, with bright color
boundD = sphere(pos=bindsite+vector(hydrolysis_sh,0,mysphererad*2), radius=mysphererad*0.9)
boundD.visible = False # initially none, True when hydrolysis occurs
# after a tRNA-GDP is rejected, briefly show it as ghost
rejectD = sphere(pos=bindsite+vector(0.5*mysphererad+hydrolysis_sh, 0.3*mysphererad, \
                                     2.2*mysphererad), radius=mysphererad*.85, opacity=myopaci)
rejectD.visible = False # initially none, True when tRNA-GDP rejection occurs
# just  before a tRNA-GTP binds, briefly show it as ghost
arrivingT = cylinder(pos=bindsite+vector(-.5*mysphererad, 0.3*mysphererad, 1.6*mysphererad), \
                     axis=vector(0,0,.17), radius=mysphererad*.80, opacity=myopaci)
arrivingT.visible = False # initially none, True when tRNA-GTP is arriving

chain = [] #list of bright spheres representing incorporated amino acids. Last item in chain is most recent 
           #addition and will be displayed leftmost.
chainsite = vector(+.25,0,.1) # leftmost position of amino acid chain
timerT = 0; timerD = 0      # initialize timer for how long rejection complexes will appear, timerT for tRNA-GTP complex, timerD for tRNA-GDP complex
frametext = label(text=str(0), color=color.green, pos=vector(0,-1.0,0.15)) # text on screen with frame and state index

<IPython.core.display.Javascript object>

In [4]:
for whichframe in range(1, int(Nframes*fracshow)+1): # here we assume the frame from m=1 to m=2 is the first frame 
                                                     # and skip the m=0 to m=1 frame 
                                                     # so that the lookback code below can go back one state at the beginning
    frametext.text = str(whichframe)+" : "+str(Jm[whichframe]) # update text displaying frame and state index
    arriving = 0; arrivingT.visible = False  # reset the arriving amino acid on each video frame
    # animate rejection, but if rejected complex has been displayed for more than 
    # rejectduration number of frames, turn off display:
    if timerD > rejectduration: rejectD.visible = False
    else: rejectD.pos += vector(0, 0.15*mysphererad*1.5, 0.15*mysphererad*1.5) # move rejected complex a little bit each frame
    if timerT > rejectduration: rejectT.visible = False
    else: rejectT.pos += vector(0, 0.15*mysphererad*1.5, 0.15*mysphererad*1.5)
    for lookahead in range(Jm[whichframe], Jm[whichframe+1]): # check if arrival in the next frame
        if states[lookahead]==0: # if current frame is displaying empty ribosome, check for arriving tRNA in next state
            if states[lookahead+1]==1: arriving=1 # 1 is for correct tRNA-GTP complex binding
            elif states[lookahead+1]==2: arriving=2 # 2 is for wrong tRNA-GTP complex binding
    for lookback in range(Jm[whichframe-1]+1, Jm[whichframe]+1): # check if rejection or incorporation in this frame
        # the +1 outside the brackets accounts for the possibility that two or no tranisiton times happened within the current frame
        if states[lookback]==0: # if current state is an empty ribosome, check for rejection in the previous state
            if states[lookback-1]==1:            # check if C.GTP was rejected
                rejectT.visible = True           # later will also turn off boundT
                timerT = 0                       # display rejected amino acid for rejectduration number of frames 
                rejectT.color = vector(.5,1,.5)  # dark translucent green
                rejectT.pos = bindsite+vector(0.5*mysphererad, 0.3*mysphererad, 1.7*mysphererad)
            elif states[lookback-1]==2:          # check if W.GTP was rejected
                rejectT.visible = True           # later will also turn off boundT
                timerT = 0                       # display rejected amino acid
                rejectT.color = vector(1,.5,.5)  # dark translucent red
                rejectT.pos = bindsite+vector(0.5*mysphererad, 0.3*mysphererad, 1.7*mysphererad)
            elif states[lookback-1]==3:          # check if C.GDP was rejected
                rejectD.visible = True           # later will also turn off boundD
                timerD = 0                       # display rejected amino acid 
                rejectD.color = vector(0,1,0)    # bright translucent green
                rejectD.pos = bindsite+vector(0.5*mysphererad+hydrolysis_sh, 0.3*mysphererad, \
                                     2.2*mysphererad)
            elif states[lookback-1]==4:          # check if W.GDP was rejected
                rejectD.visible = True           # later will also turn off boundD
                timerD = 0                       # display rejected amino 
                rejectD.color = vector(1,0,0)    # bright translucent red
                rejectD.pos = bindsite+vector(0.5*mysphererad+hydrolysis_sh, 0.3*mysphererad, \
                                     2.2*mysphererad)
            elif states[lookback-1]==-3: # check if correct amino acid was incorporated
                # this is not a rejection so rejectD is not turned on, but if it's already on from a
                # previous rejection event leave it on
                for amino in chain:              # push all amino acids to right
                    amino.pos = amino.pos + vector(mysphererad*1.9,0,0)
                chain = chain + [sphere(pos=chainsite+vector(0,0,.3), \
                                        radius=mysphererad*0.9,color=vector(0,1,0))]
            elif states[lookback-1]==-4:# check if wrong amino acid was incorporated
                # this is not a rejection so rejectD is not turned on, but if it's already on from a
                # previous rejection event leave it on
                for amino in chain:              # push all amino acids to right
                    amino.pos = amino.pos + vector(mysphererad*2,0,0)
                chain = chain + [sphere(pos=chainsite+vector(0,0,.3), \
                                        radius=mysphererad*0.9,color=vector(1,0,0))]
    newstate = states[Jm[whichframe]] # display state that is current at the beginning of this frame
    if newstate==0:                        # unoccupied; continue to display rejected complex if there is one
        boundT.visible = False
        boundD.visible = False
    elif newstate==1:                      # correct tRNA.GTP: dark green
        boundT.visible = True
        boundT.color = vector(0,.3,0)
        boundD.visible = False
    elif newstate==2:                      # wrong tRNA.GTP: dark red
        boundT.visible = True
        boundT.color = vector(.3,0,0)
        boundD.visible = False
    elif newstate==3 or newstate==-3:      # correct tRNA.GDP: green
        boundT.visible = False
        boundD.visible = True
        boundD.color = vector(0,1,0)
    elif newstate==4 or newstate==-4:      # wrong tRNA.GDP: red
        boundT.visible = False
        boundD.visible = True
        boundD.color = vector(1,0,0)
    # animate arrival:
    if arriving==1: 
        arrivingT.visible = True; arrivingT.color = vector(.5,1,.5)
    if arriving==2:
        arrivingT.visible = True; arrivingT.color = vector(1,.5,.5)
    timerT += 1; timerD += 1 # increment timer for rejection complexes
    if povray: 
        povexport.export(canvas=scene, filename=outfile_name.format(whichframe))
    rate(framerate)                        # pause till it's time to show next frame

correct incorporation frame  41 , state index= 30
